In [1]:
%pip install -U transformers --user
%pip install packaging --user
#%pip install flash-attn --user
# %pip install bitsandbytes --user
%pip install https://github.com/jllllll/bitsandbytes-windows-webui/releases/download/wheels/bitsandbytes-0.41.2.post2-py3-none-win_amd64.whl --user
%pip install accelerate --user
%pip install -U python-dotenv BeautifulSoup4 playwright ipywidgets --user
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --user
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu113 --user
%pip install pandas --user

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/152.7 MB ? eta -:--:--
     -------------------------------------- 0.0/152.7 MB 640.0 kB/s eta 0:03:59
     ---------------------------------------- 0.1/152.7 MB 1.8 MB/s eta 0:01:26
     ---------------------------------------- 0.4/152.7 MB 3.4 MB/s eta 0:00:46
     ---------------------------------------- 0.8/152.7 MB 4.6 MB/s eta 0:00:34
     ---------------------------------------- 1.2/152.7 MB 5.4 MB/s eta 0:00:29
     ---------------------------------------- 1.6/152.7 MB 6.2 MB/s eta 0:00:25
      --------------------------------------- 2.1/152.7 MB 6.8 MB/s eta 0:00:23
      --------------------------------------- 2.7/152.7 MB 7.5 MB/s eta 0:00:21
      --------------------------------------- 3.3/152.7 MB 8.1 MB/s eta 0:00:19
     - -------------------------------------- 4.1/152.7 MB 8.9 MB/s eta 0:0

In [1]:
# Environment setup
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Get the access token
access_token = os.getenv("HUGGINGFACE_TOKEN")

In [2]:
# This is a helper function to get a dynamic page's HTML content using Playwright, and then parse it with BeautifulSoup.
# Necessary for pages that are rendered with JavaScript
from playwright.sync_api import sync_playwright
from bs4 import BeautifulSoup

def get_dynamic_soup(url: str) -> BeautifulSoup:
    with sync_playwright() as p:
        browser = p.chromium.launch()
        page = browser.new_page()
        page.goto(url)
        soup = BeautifulSoup(page.content(), "html.parser")
        browser.close()
        return soup

In [3]:
# import ./data/endpoints.csv with rows URL,Text
import pandas as pd
endpoints = pd.read_csv('./data/endpoints.csv')

# remove any rows with empty URL or Text
endpoints = endpoints.dropna(subset=['URL', 'Text'])

#filter out urls with search term in the text ex. ?e=, ?q=, ?s=, ?search=
filter_out_terms = ['?']
# Create a mask that is True for rows where the 'Text' column contains any of the filter_out_terms
mask = endpoints['URL'].apply(lambda x: any(term in x for term in filter_out_terms))
# Use ~ to negate the mask, i.e., keep only the rows where the mask is False
endpoints = endpoints[~mask]

#sort endpoints by how many paths they have ex. None, /path, /path/path, /path/path/path
# get the number of paths in a url
def get_path_count(url: str) -> int:
    return url.count('/') - 2
endpoints['path_count'] = endpoints['URL'].apply(get_path_count)
endpoints = endpoints.sort_values(by=['path_count'])

# endpoints where path_count is 0
subdomain_test_set = endpoints[endpoints['path_count'] == 0]

# Ensure "Text" column is a string
subdomain_test_set['Text'] = subdomain_test_set['Text'].astype(str)

# Remove any rows with "Text" less than 50 characters
subdomain_test_set = subdomain_test_set[subdomain_test_set['Text'].str.len() > 50]

#sort by length of URL to get the shortest first
# Create a new column 'URL_length' that contains the length of each URL
subdomain_test_set['URL_length'] = subdomain_test_set['URL'].str.len()
# Sort by 'URL_length' to get the shortest URLs first
subdomain_test_set = subdomain_test_set.sort_values(by=['URL_length'])
# If you don't want to keep the 'URL_length' column, you can drop it
subdomain_test_set = subdomain_test_set.drop(columns=['URL_length'])

# Remove any rows with duplicate URLs
subdomain_test_set = subdomain_test_set.drop_duplicates(subset=['URL'])

# save test set to csv
subdomain_test_set.to_csv('./data/subdomain_test_set.csv', index=False)

# get the first 30 rows
subdomain_test_set.head(100)

# convert the test set to a list of tuples (URL, Text)
test_set = list(subdomain_test_set.itertuples(index=False, name=None))

C:\Users\Ryan\AppData\Local\Temp\ipykernel_18516\2046674061.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subdomain_test_set['Text'] = subdomain_test_set['Text'].astype(str)


In [4]:
# LLM Summary Generation
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, AutoConfig, BitsAndBytesConfig
from textwrap import dedent

MODEL = "google/gemma-2b-it" # Newer and small model, promising given the size to performance as well as open source
# MODEL = "yam-peleg/Experiment26-7B" # too large for my GPU

device = "cuda:0" if torch.cuda.is_available() else "cpu"
#device = "cpu"
print(device)

config = AutoConfig.from_pretrained(MODEL, use_auth_token=access_token, max_new_tokens=512)
quantization_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)

model = AutoModelForCausalLM.from_pretrained(MODEL, use_auth_token=access_token, config=config, device_map=device)#.to(device) #.to not needed if quantization is used, defaults to cuda
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_auth_token=access_token, config=config)

generator = pipeline("text-generation", model=MODEL, config=config)

cuda:0


C:\Users\Ryan\AppData\Roaming\Python\Python312\site-packages\transformers\models\auto\configuration_auto.py:1096: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
C:\Users\Ryan\AppData\Roaming\Python\Python312\site-packages\transformers\models\auto\auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


bin C:\Users\Ryan\AppData\Roaming\Python\Python312\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.
C:\Users\Ryan\AppData\Roaming\Python\Python312\site-packages\transformers\models\auto\tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
import json

LLM_INSTRUCT_PROMPT = dedent('''\
Given the URL "{url}", and the following text from the site: "{text}" provide a concise summary that includes: 
1. The main topics in the text.
2. The purpose or objective of the website, inferred from the text and url (including subdomain and path).
3. Tags or keywords that a user may search to try and find the site in a search engine.

You must utilize information from the URL (such as the specific path and subdomain) to contextualize and add to the understanding of the text.

Format the response in a json like the following:

'summary': 'The description of url and summary of the text goes here.',
'topics': ['topic1', 'topic2', 'topic3'],
'tags': ['tag1', 'tag2', 'tag3']
''')

# This function takes a prompt and returns generated text with transformers generator
def generate_summary(url: str, text: str, debug: bool = False) -> str:
    prompt = LLM_INSTRUCT_PROMPT.format(url=url, text=text)
    return generator(prompt, max_length=800, do_sample=True)

# Manual
def gen_summary(url: str, text:str, debug: bool = False) -> str:
    chat = [
        { "role": "user", "content": LLM_INSTRUCT_PROMPT.format(url=url, text=text) },
    ]
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer([prompt], add_special_tokens=False, return_tensors="pt").to(device)
    output = model.generate(**input_ids, max_new_tokens=1000)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Helpers
def fill_prompt(input_text: str) -> str:
    return LLM_INSTRUCT_PROMPT + " " + input_text

def extract_summary(llm_output: str):
    model_index = llm_output.find("'tags': ['tag1', 'tag2', 'tag3']\nmodel")
    if model_index != -1:
        json_string = llm_output[model_index + 40:]
        
        # Replace ",] with "] to fix json formatting, common issue with LLM
        json_string = json_string.replace('",]', '"]')
        
        # If last character is not } then add it
        if json_string[-1] != '}':
            json_string += '}'
            
        # If first character is not { then add it
        if json_string[0] != '{':
            json_string = '{' + json_string
        
        try:
            json_object = json.loads(json_string)
            return json_object
        except:
            print('Error parsing json: ' + json_string)
            return None
    else:
        print('model not found in string')

In [14]:
# Test llm summaries
summary_outputs = []
extract_summaries = []
final_urls = []

for t in test_set:
    try:
        sum = gen_summary(t[0],t[1])
        summary_outputs.append(sum)
        extract = extract_summary(sum)
        extract_summaries.append(extract)
        if extract is not None:
            extract['url']=t[0]
            print(extract)
            final_urls.append(extract)
    except Exception as e:
        print(e)
        print('Error with URL:', t[0])
        continue
            
print(final_urls)
    

{'summary': 'The University of North Carolina at Charlotte is a top-ranked institution with a wide range of programs and resources available to students. The website provides a comprehensive overview of the university, including news, events, research, and academic programs. It also offers information on how to apply to the university, including admissions requirements and financial aid options.', 'topics': ['University', 'News & Events', 'Research', 'Academic Programs'], 'tags': ['UNC Charlotte', 'University', 'North Carolina', 'Education', 'Research', 'Career', 'Student Life'], 'url': 'https://charlotte.edu'}
{'summary': "This website provides information about the Health Informatics and Analytics Master's program at UNC Charlotte, a program designed to transform the quality and efficiency of healthcare through data.", 'topics': ['Health Informatics and Analytics', 'Master of Health Informatics and Analytics'], 'tags': ['HIA', "Master's", 'Data Science'], 'url': 'http://hi.charlotte.

In [15]:
# Save the final urls to a json file
with open('./data/llm_summaries.json', 'w') as f:
    json.dump(final_urls, f, indent=4)

In [ ]:
# Clear llm model from memory

del model
del tokenizer
del generator


In [16]:
import pprint

# If final_urls is none, import the json file

with open('./data/llm_summaries.json', 'r') as f:
    final_urls = json.load(f)

pprint.pprint(final_urls)

[{'summary': 'The University of North Carolina at Charlotte is a top-ranked '
             'institution with a wide range of programs and resources '
             'available to students. The website provides a comprehensive '
             'overview of the university, including news, events, research, '
             'and academic programs. It also offers information on how to '
             'apply to the university, including admissions requirements and '
             'financial aid options.',
  'tags': ['UNC Charlotte',
           'University',
           'North Carolina',
           'Education',
           'Research',
           'Career',
           'Student Life'],
  'topics': ['University', 'News & Events', 'Research', 'Academic Programs'],
  'url': 'https://charlotte.edu'},
 {'summary': 'This website provides information about the Health Informatics '
             "and Analytics Master's program at UNC Charlotte, a program "
             'designed to transform the quality and effic

# Embedding Vector Search

In [17]:
# %pip install txtai --user

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


Note: you may need to restart the kernel to use updated packages.


In [19]:
# Embedding model for similarity
## Txtai
import txtai
    
# Create embeddings for each web page
embeddings = txtai.Embeddings(path="sebastian-hofstaetter/distilbert-dot-tas_b-b256-msmarco", content=True)

def to_displayable_text(row):
    return f"{row['url']}\n{row['summary']}\n{row['topics']}\n{row['tags']}"

if final_urls is not None:
    embeddings.index([(row["url"], to_displayable_text(row)) for row in final_urls])


In [33]:
results = embeddings.search("computer science masters program requirements", 2)
print("computer science masters program requirements")
for result in results:
    print(result['id'])

results = embeddings.search("football team", 2)
print("football team")
for result in results:
    print(result['id'])
    
    
print("49er sports "+str(embeddings.search("49er sports", 2)))    
    
embeddings.explain("computer science", limit=1)

computer science masters program requirements
https://mathfinance.charlotte.edu
https://cci.charlotte.edu
football team
https://haltonarena.charlotte.edu
https://geoearth.charlotte.edu
49er sports [{'id': 'https://democracyexperience.charlotte.edu', 'text': "https://democracyexperience.charlotte.edu\nThe 49er Democracy Experience is a dynamic platform for learning about and engaging in civic life at the University of North Carolina at Charlotte. It offers voter registration, primary and election news, volunteer opportunities, and more, empowering students and community members to participate in shaping a productive society.\n['Civic Engagement', 'Voter Education', 'Leadership']\n['49er Democracy Experience', 'UNC Charlotte', 'Civic Engagement', 'Voter Registration']", 'score': 0.7356357574462891}, {'id': 'https://49eralumni.charlotte.edu', 'text': "https://49eralumni.charlotte.edu\nThe 49er Alumni Campus Employees Network (ACE) is a community hub for alumni, students, and faculty. The 

[{'id': 'https://cci.charlotte.edu',
  'text': "https://cci.charlotte.edu\nThe College of Computing and Informatics is a leading institution for computing and informatics education. It offers undergraduate and graduate programs in computer science, cybersecurity, information technology, and other related fields.\n['Computer Science', 'Cybersecurity', 'Information Technology']\n['College of Computing and Informatics', 'University of North Carolina at Charlotte', 'Computer Science Education']",
  'score': 0.8016824722290039,
  'tokens': [('https://cci.charlotte.edu', -0.010378360748291016),
   ('The', 0.0017098188400268555),
   ('College', -0.005206465721130371),
   ('of', -0.0015467405319213867),
   ('Computing', 0.004706025123596191),
   ('and', -0.002369105815887451),
   ('Informatics', 0.0028295516967773438),
   ('is', 0.0029227733612060547),
   ('a', 0.0019158124923706055),
   ('leading', -0.002102375030517578),
   ('institution', 0.0015689730644226074),
   ('for', -0.00032877922058

# To do:

* Add more urls and pages to test dataset
* Export variables to files for easier importation
* Create python api wrapper for txtai searches
* Try other vector search models and embeddings 